In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed

In [2]:
# モデルのロード
model_name = "sbintuitions/sarashina2.2-3b-instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
chat_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
set_seed(123)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
# ユーザーの入力
user_input = [{"role": "user", "content": "こんにちは。あなたの名前を教えて"}]

In [4]:
# モデルによる応答生成
responses = chat_pipeline(
    user_input,
    max_length=50,
    do_sample=True,
    num_return_sequences=3,
)

Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [5]:
# 応答を表示
for i, response in enumerate(responses, 1):
    print(f"Response {i}: {response['generated_text']}")

Response 1: [{'role': 'user', 'content': 'こんにちは。あなたの名前を教えて'}, {'role': 'assistant', 'content': 'Sarashina2と言います。本日のご要件を教えて下さい。'}]
Response 2: [{'role': 'user', 'content': 'こんにちは。あなたの名前を教えて'}, {'role': 'assistant', 'content': 'Sarashina2と言います。本日はどんなご用件でしょうか？お手伝いできることがあれば教えてください。'}]
Response 3: [{'role': 'user', 'content': 'こんにちは。あなたの名前を教えて'}, {'role': 'assistant', 'content': 'Sarashina2と言います。本日のご要件を教えて下さい。'}]


In [6]:
user_input = [
    {"role": "system", "content": "ユーザの発言を平易化し，さらに書き言葉から話し言葉に変換してください。"},
    {"role": "user", "content": "千葉工業大学は千葉県習志野市にある工業大学である。"}
]

In [7]:
responses = chat_pipeline(
    user_input,
    max_length=50,
    do_sample=True,
    num_return_sequences=3,
)

Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [8]:
for i, response in enumerate(responses, 1):
    print(f"Response {i}: {response['generated_text']}")

Response 1: [{'role': 'system', 'content': 'ユーザの発言を平易化し，さらに書き言葉から話し言葉に変換してください。'}, {'role': 'user', 'content': '千葉工業大学は千葉県習志野市にある工業大学である。'}, {'role': 'assistant', 'content': '千葉工業大学は、千葉県習志野市にある工業大学ですね。engineeringに特化した大学として知られています。もし何か質問や詳しく知りたいことがあれば、お気軽にお尋ねください！'}]
Response 2: [{'role': 'system', 'content': 'ユーザの発言を平易化し，さらに書き言葉から話し言葉に変換してください。'}, {'role': 'user', 'content': '千葉工業大学は千葉県習志野市にある工業大学である。'}, {'role': 'assistant', 'content': '千葉工業大学は、千葉県習志野市にある工業大学ですね。簡単に説明すると、この大学は千葉県にあり、特に工学に関する教育や研究に力を入れている学校です。'}]
Response 3: [{'role': 'system', 'content': 'ユーザの発言を平易化し，さらに書き言葉から話し言葉に変換してください。'}, {'role': 'user', 'content': '千葉工業大学は千葉県習志野市にある工業大学である。'}, {'role': 'assistant', 'content': '千葉工業大学は、千葉県習志野市にある工業大学ですね。もっと簡単に言うと、この大学は千葉県にあり、特に技術に関する勉強ができる学校です。'}]


In [12]:
input_text = tokenizer.apply_chat_template(user_input, tokenize=False)

In [33]:
input_text

'<|system|>ユーザの発言を平易化し，さらに書き言葉から話し言葉に変換してください。</s><|user|>千葉工業大学は千葉県習志野市にある工業大学である。</s>'

In [13]:
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [30]:
len(inputs.input_ids[0])

29

In [31]:
outputs_removed_input = [output[len(inputs.input_ids[0]):] for output in outputs]

In [32]:
tokenizer.batch_decode(outputs_removed_input, skip_special_tokens=True)

['\n（平易化・話し言葉に変換）\n\n千葉工業大学は、千葉の習志野にある工業系の大学だよ。簡単に言うと、工学を学ぶための大学なんだ。場所は、千葉県の習志野ってところにあって、分かりやすい場所にあるよ。」',
 '\n（平易化：千葉工業大学は、千葉県の習志野市にある工業大学です。話し言葉にすると：千葉工大は、千葉の習志野にある工業系の大学なんだ。）',
 '\n（平易化と話し言葉への変換をお願いします）']